# Cross-validation and hyperparameters

In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_regression

X, y = make_regression(n_samples=100, n_features=100, n_informative=10)

# Vanilla LR Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X, y)
#print(lr.intercept_, lr.coef_)
preds = lr.predict(X) #predict

print(np.sqrt(mean_squared_error(preds, y)))

# Did we overfit?

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Create the CV iterator
cv_iterator = KFold(n_splits=5, shuffle=True, random_state=10)

# Note: default in sklearn: higher return values are better than lower return values
cross_val_score(lr, X, y, cv=cv_iterator, scoring="neg_mean_squared_error")
#cross_val_score(lr, X, y, cv=5, scoring="neg_mean_squared_error")
#abs(np.mean(cross_val_score(lr, X, y, cv=5, scoring="neg_mean_squared_error")))

# Let's plot some learning curves

In [ ]:
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
title = "Learning Curves"

# Create the CV iterator
cv_iterator = KFold(n_splits=5, shuffle=True, random_state=10)
lr = LinearRegression()

plot_learning_curve(lr, title, X, y, cv=cv_iterator, n_jobs=4)

plt.show()

# With Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

llr = Lasso(alpha=0.5)
llr.fit(X, y)
#print(lr.intercept_, lr.coef_)
preds = llr.predict(X) #predict

print(np.sqrt(mean_squared_error(preds, y)))

# Create the CV iterator
cv_iterator = KFold(n_splits=5, shuffle=True, random_state=10)

# Note: default in sklearn: higher return values are better than lower return values
cross_val_score(llr, X, y, cv=cv_iterator, scoring="neg_mean_squared_error")
cross_val_score(llr, X, y, cv=5, scoring="neg_mean_squared_error")
abs(np.mean(cross_val_score(llr, X, y, cv=5, scoring="neg_mean_squared_error")))

In [ ]:
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
title = "Learning Curves"

# Create the CV iterator
cv_iterator = KFold(n_splits=5, shuffle=True, random_state=10)
llr = Lasso(alpha=0.5)

plot_learning_curve(llr, title, X, y, cv=cv_iterator, n_jobs=4)

plt.show()

# Nested GridSearchCV

In [ ]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error

# Parameter grid
p_grid = {
    "alpha": [0.1, 0.5, 1, 1.5]
}

# CV iterators
inner_cv_iterator = KFold(n_splits=5, shuffle=True, random_state=10)
outer_cv_iterator = KFold(n_splits=5, shuffle=True, random_state=10)

# Hyperparameter optimization
lasso = Lasso()
llr = GridSearchCV(estimator=lasso, param_grid=p_grid, cv=inner_cv_iterator)
cross_val_score(llr, X=X, y=y, cv=outer_cv_iterator)